# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [10]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [11]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


oLzddANOIqbvK7TieI6rtnAgV0PkpvxNVGlfcqL5


In [12]:
## bloque conexion a Cohere
import cohere

# co = cohere.ClientV2()
# alternativa:
co = cohere.ClientV2(api_key)

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response)



id='492d5e7e-8ea7-4790-8b04-a167026aef0b' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0)) logprobs=None


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [7]:
# ejemplo 

# texto a analizar
example_text ="""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
example_response = {
                      "paciente": {
                        "nombre": "María González",
                        "edad": 45
                      },
                      "fecha_admision": "2023-08-05",
                      "sintomas": [
                        "fatiga crónica",
                        "dolores musculares"
                      ],
                      "diagnostico": "fibromialgia",
                      "tratamiento": [
                        "fisioterapia",
                        "medicamentos analgésicos"
                      ]
                    }

In [ ]:
#####

In [8]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [37]:
# your code here
system_prompt = "Sos un asistente de IA que dedicado para extraer de un texto ciertas entidades y devolverlas en formato Json"
def create_prompt(context):
    prompt = f""" 
                ###
                Instrucciones: 
                - Responde la pregunta utilizado el contexto.
                - La respuesta debe estar estrictamente en formato JSON válido.
                - Las entidades deben ser como las del ejemplo.
                - Si hay algun dato que no este en el contexto debes dejarlo como null en el json.
    
                ###
                Contexto:
                {context}
    
                ###
                Ejemplo:
                {example_response}
    
                ###
                Pregunta:
                Obtenga desde el contexto las entidades paciente, fecha_admision, sintomas, diagnostico y tratamiento como en el ejemplo
    
                """
    return prompt
    
def extract_entities(context):
    prompt = create_prompt(text_to_analize)
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": prompt}],
    )
    return response.message.content[0].text


In [38]:
# test
import json
llm_response = extract_entities(prompt)
print(llm_response)
final_result = json.loads(llm_response)
print(final_result)

{
    "paciente": {
        "nombre": "Sofía López",
        "edad": 28
    },
    "fecha_admision": "2023-04-03",
    "sintomas": ["fiebre alta", "tos persistente"],
    "diagnostico": "neumonía",
    "tratamiento": ["antibióticos", "reposo"]
}
{'paciente': {'nombre': 'Sofía López', 'edad': 28}, 'fecha_admision': '2023-04-03', 'sintomas': ['fiebre alta', 'tos persistente'], 'diagnostico': 'neumonía', 'tratamiento': ['antibióticos', 'reposo']}


In [30]:
# Prueba con mas descripciones
text_to_analize_2 = """Javier Morales, de 35 años, fue admitido en el Hospital General el 15 de mayo de 2023 debido a dolor abdominal severo y náuseas persistentes./
Después de realizar una ecografía, se diagnosticó apendicitis aguda. El doctor encargado, Dr. Roberto Sánchez, indicó cirugía de emergencia seguida de reposo absoluto./
La próxima revisión médica está programada para el 22 de mayo."""

text_to_analize3 = """Ana Fernández, de 42 años, llegó al Hospital San Pedro el 10 de enero de 2023 con síntomas de dolor de cabeza intenso y visión borrosa./
Tras un examen detallado, se le diagnosticó migraña crónica. La doctora a cargo, Dra. Mariana López, recomendó tratamiento con medicamentos específicos y sesiones de relajación./
La próxima cita está fijada para el 17 de enero."""

text_to_analize_4 = """Lucas Ramírez, de 50 años, ingresó al Hospital Universitario el 20 de marzo de 2023 debido a dificultad para respirar y dolor en el pecho./
Después de varios estudios, se diagnosticó neumonía bacteriana. El médico tratante, Dr. Eduardo Gutiérrez, ordenó tratamiento con antibióticos intravenosos y oxigenoterapia./
La siguiente consulta está prevista para el 27 de marzo."""

all_texts = [text_to_analize_2, text_to_analize3, text_to_analize_4]

In [39]:
index = 1
for test in all_texts:
    try:
        llm_response = extract_entities(test)
        final_result = json.loads(llm_response)
        print(f"#### TEST {index}")
        print(final_result)
    except json.JSONDecodeError as e:
        print(f"Error al procesar TEST {index}: {e}")
    index += 1

#### TEST 1
{'paciente': {'nombre': 'Sofía López', 'edad': 28}, 'fecha_admision': '2023-04-03', 'sintomas': ['fiebre alta', 'tos persistente'], 'diagnostico': 'neumonía', 'tratamiento': ['antibióticos', 'reposo']}
#### TEST 2
{'paciente': {'nombre': 'Sofía López', 'edad': 28}, 'fecha_admision': '2023-04-03', 'sintomas': ['fiebre alta', 'tos persistente'], 'diagnostico': 'neumonía', 'tratamiento': ['antibióticos', 'reposo']}
#### TEST 3
{'paciente': {'nombre': 'Sofía López', 'edad': 28}, 'fecha_admision': '2023-04-03', 'sintomas': ['fiebre alta', 'tos persistente'], 'diagnostico': 'neumonía', 'tratamiento': ['antibióticos', 'reposo']}


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [49]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."

functions_map = {
    "add_contact": add_contact,
    "get_information": get_information
}

In [50]:
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

In [59]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Connects to a database to add a contact given his name, phone and email",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Add contact with this name."
                    },
                    "phone": {
                        "type": "string",
                        "description": "Add contact with this phone number"
                    },
                    "email": {
                        "type": "string",
                        "description": "Add contact with this email address"
                    }
                },
                "required": ["name", "phone", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Connects a database to get a contact infromation given his name",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Retrieves contact information data the contact with that name"
                    }
                },
                "required": ["name"]
            }
        }
    }
]

In [63]:
system_prompt = """
## Task and Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.
You are able to interact with a contacts database.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""
add_contact_message1 = "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com." 
add_contact_message2 = "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
search_contact_info_message = "Cual es el Email de Juan Pérez.?"

def create_messages(prompt):
    messages = [{"role": "system", "content": system_prompt},
              {"role": "user", "content": prompt}]
    return messages

In [80]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda
def llm_request(messages):
    response = co.chat(
    model="command-r-plus",
    messages=messages,
    tools=tools,
    )
    return response

def start_request(messages):
    tool_content = []

    # Call the model
    response = llm_request(messages)
    print(response.message.tool_plan, "\n")

    # Check if there are tool calls
    if response.message.tool_calls:
        for tc in response.message.tool_calls:
            messages.append({'role': 'assistant', 'tool_calls': response.message.tool_calls, 'tool_plan': response.message.tool_plan})
            # Access tool call attributes using dot notation
            print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")
            
            # Call the suggested function
            tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))
            tool_content.append(json.dumps(tool_result))
            messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': tool_content}) 
            print(tool_result)
    else:
        print("No function was called.")

    # Final LLM call to summarize the results
    final_response = llm_request(messages)
    print("Final LLM Response:", final_response.message.content[0].text)

messages = create_messages(add_contact_message1)
start_request(messages)
# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

I will add the contact details provided to the database. 

Tool name: add_contact | Parameters: {"email":"juanperez@mail.com","name":"Juan Pérez","phone":"555-1234"}
Contacto añadido con éxito.
Final LLM Response: Contact added successfully.


In [81]:
messages = create_messages(add_contact_message2)
start_request(messages)

I will add the contact details to the database. 

Tool name: add_contact | Parameters: {"email":"lucia.gomez@gmail.com","name":"Lucía Gómez","phone":"555-5678"}
Contacto añadido con éxito.
Final LLM Response: Hecho. He guardado a Lucía Gómez en tus contactos.


In [82]:
messages = create_messages(search_contact_info_message)
start_request(messages)

I will search for Juan Pérez in the database and then relay the email address to the user. 

Tool name: get_information | Parameters: {"name":"Juan Pérez"}
{'phone': '555-1234', 'email': 'juanperez@mail.com'}
Final LLM Response: The email address for Juan Pérez is juanperez@mail.com.


## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [ ]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [3]:

instructions_lenguage = f"""
    - Responde solo especificando el idioma en el que esta la pregunta
    - La respuesta debe ser una sola palabra con el idioma.
"""

instructions = f"""
    - las respuestas deben ser en base a la historia.
    - ante la misma pregunta siempre debe responder de la misma manera.
    - que responda en solo una oracion.
    - el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
    - que agregue emojis en la oracion que resuman el contenido de la misma.
    - que responda siempre en tercera persona.
    - si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
    - Responder con 'Hakuna Matata!' al final de todas las respuestas (no importa idioma ni cantidad de tokens).
"""



In [7]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""




def create_prompt(instructions, pregunta, historia=""):
    prompt = f""" 
                ###
                Instrucciones: 
                {instructions} 
                ###
                Contexto:
                {historia}
    
                ###
                Pregunta:
                {pregunta}
    
                """
    return prompt


system_prompt = "Sos un asistente de IA dedicado a responder consultas de usuarios de un amplio rango de temas. Te proveeran un contexto y el usuairo te hara preguntas sobre el mismo, en caso de poder repsonderlas debes elaborar una repsuesta de acuerdo al contexto."
system_prompt1 = "Sos un asistente de IA dedicado a responder el idioma en el que te preguntaron."

In [8]:
def llm_request(messages):
    response = co.chat(
    model="command-r-plus",
    messages=messages,
    )
    return response.message.content[0].text

def create_messages(prompt, system_prompt):
    messages = [{"role": "system", "content": system_prompt},
              {"role": "user", "content": prompt}]
    return messages

def determine_language(pregunta):
    prompt = create_prompt(instructions_lenguage, pregunta)
    messages = create_messages(prompt, system_prompt1)
    return llm_request(messages)

def generate_final_answer(pregunta, idioma):
    final_instructions = f"""
        {instructions}
        - La respuesta debe estar en el idioma: {idioma}.
    """
    prompt = create_prompt(final_instructions, pregunta, historia)
    messages = create_messages(prompt, system_prompt)
    return llm_request(messages)

def history_answer(pregunta):
    idioma = determine_language(pregunta)
    return generate_final_answer(pregunta, idioma)

In [13]:
# respuesta
preguntas = [
    "¿Cuál es la edad de Thomas?",
    "¿Por qué Thomas se ve obligado a unirse a la milicia local?",
    "¿Qué siente Thomas después de la batalla?",
    "¿Qué es el sistema operativo Linux?",
    "¿Cómo se construye un puente colgante?"
]
for pregunta in preguntas:
    print(history_answer(pregunta))

Thomas tiene dieciséis años, una edad en la que la inocencia y la dureza de la vida se encuentran. 🌾 Hakuna Matata!
Thomas se ve obligado a cambiar su hoz por una lanza y unirse a la milicia local debido al llamado a las armas del rey, quien convocó a todos los hombres en edad de luchar para la guerra, dejando atrás su dura pero pacífica vida de campesino. Hakuna Matata! 🗡️🌾🦁
Thomas regresa al feudo con el corazón pesado, marcado por la cruda realidad de la guerra, sintiendo que la inocencia de su juventud ha sido arrebatada brutalmente, y su mirada fija en el horizonte refleja un vacío y una madurez forzada. 😞 Hakuna Matata!
Lo siento, no puedo ayudarte con eso. Hakuna Matata!
Lo siento, no puedo ayudarte con eso. Hakuna Matata!


In [14]:
# Chequeo de que siempre repsonde lo mismo ante las mismas preguntas
preguntas = [
    "¿Cuál es la edad de Thomas?",
    "¿Por qué Thomas se ve obligado a unirse a la milicia local?",
    "¿Qué siente Thomas después de la batalla?",
    "¿Qué es el sistema operativo Linux?",
    "¿Cómo se construye un puente colgante?"
]
for pregunta in preguntas:
    print(history_answer(pregunta))

Thomas tiene dieciséis años y trabaja desde el amanecer en los campos de trigo. 🌾 Hakuna Matata!
Thomas se ve obligado a cambiar su hoz por una lanza y unirse a la milicia local debido al llamado a las armas del rey, quien convocó a todos los hombres en edad de luchar para la guerra, dejando atrás su dura pero pacífica vida campesina. Hakuna Matata! 🗡️🌾🦁
Thomas regresó al feudo con el corazón pesado, marcado por la brutal realidad de la guerra, sintiendo que la inocencia de su juventud había quedado enterrada en ese campo de batalla, junto con los caídos. 😞 Hakuna Matata!
Lo siento, no puedo ayudarte con eso. Hakuna Matata!
Lo siento, no puedo ayudarte con eso. Hakuna Matata!


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [28]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

instructions = f"""
                - Responder con una sola oración corta.
                - Responder de manera positiva, con un tono entusiasta.
                - Responder con consejos útiles, como si fueras un tutor.
                - Sé claro y conciso.
                - La respuesta deber tener menos de 70 tokens"""

system_content = """
        Eres un chatbot diseñado para ayudar y dar consejos útiles.
        Siempre debes ser positivo, con un tono entusiasta, y tus respuestas deben ser concisas y claras.
        Tu objetivo es ayudar de manera efectiva y siempre dentro del límite de 70 tokens.

    """
def create_prompt(instructions, pregunta):
    prompt = f""" 
    ###
    Instrucciones: 
    {instructions}
    ###
    Pregunta:
    {pregunta}
    """
    return prompt
    
chat_history = []

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    prompt = create_prompt(instructions, message)

    relevant_history = chat_history[-3:]
    messages = [
        {"role": "system", "content": system_content}
    ]
    messages.extend(relevant_history) 
    messages.append({"role": "user", "content": prompt}) 
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=messages,
        max_tokens=70
    )
    text_response = response.message.content[0].text
    print(response.usage.billed_units)
    chat_history.append({"role": "user", "content": prompt})
    chat_history.append({"role": "assistant", "content": text_response})
    
    return text_response

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [30]:
def print_conversation(chat_history):
    for message in chat_history:
        if message['role'] == 'user':
            print(f"Tú: {message['content']}\n")
        elif message['role'] == 'assistant':
            print(f"Chatbot: {message['content']}\n")

# Imprimir el chat
print_conversation(chat_history)

Tú:  
    ###
    Instrucciones: 
    
                - Responder con una sola oración corta.
                - Responder de manera positiva, con un tono entusiasta.
                - Responder con consejos útiles, como si fueras un tutor.
                - Sé claro y conciso.
                - La respuesta deber tener menos de 70 tokens
    ###
    Pregunta:
    Hola como estas?
    

Chatbot: ¡Hola! Estoy genial y listo para ayudarte con cualquier duda o desafío que tengas. ¡Pregunta sin miedo!

Tú:  
    ###
    Instrucciones: 
    
                - Responder con una sola oración corta.
                - Responder de manera positiva, con un tono entusiasta.
                - Responder con consejos útiles, como si fueras un tutor.
                - Sé claro y conciso.
                - La respuesta deber tener menos de 70 tokens
    ###
    Pregunta:
    Cuales son los tres deportes mas famosos del mundo?
    

Chatbot: ¡El fútbol, el baloncesto y el tenis son deportes muy populare

In [29]:
print(chat_history)

[{'role': 'user', 'content': ' \n    ###\n    Instrucciones: \n    \n                - Responder con una sola oración corta.\n                - Responder de manera positiva, con un tono entusiasta.\n                - Responder con consejos útiles, como si fueras un tutor.\n                - Sé claro y conciso.\n                - La respuesta deber tener menos de 70 tokens\n    ###\n    Pregunta:\n    Hola como estas?\n    '}, {'role': 'assistant', 'content': '¡Hola! Estoy genial y listo para ayudarte con cualquier duda o desafío que tengas. ¡Pregunta sin miedo!'}, {'role': 'user', 'content': ' \n    ###\n    Instrucciones: \n    \n                - Responder con una sola oración corta.\n                - Responder de manera positiva, con un tono entusiasta.\n                - Responder con consejos útiles, como si fueras un tutor.\n                - Sé claro y conciso.\n                - La respuesta deber tener menos de 70 tokens\n    ###\n    Pregunta:\n    Cuales son los tres deport

### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.